# KMC 3D확장 후로젝트

# Algorithm
---

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [3]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
'''
bond_energy = 2.07 
temperature = 700
e0 = 0.1

# site energy, e_(bond number)
e_site = np.zeros(6)
for i in range(6):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((6, 6))
for i in range(6):
    for j in range(6):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((6, 6))
for i in range(6):
    for j in range(6):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   


In [4]:
# value check
print(e_site)
print('-----------------')
print(e_a)
print('-----------------')
print(diffusion_rate)


[ 0.    -1.035 -2.07  -3.105 -4.14  -5.175]
-----------------
[[ 1.0000e-01 -3.5000e-03 -1.0700e-01 -2.1050e-01 -3.1400e-01 -4.1750e-01]
 [ 1.2385e+00  1.0000e-01 -3.5000e-03 -1.0700e-01 -2.1050e-01 -3.1400e-01]
 [ 2.3770e+00  1.2385e+00  1.0000e-01 -3.5000e-03 -1.0700e-01 -2.1050e-01]
 [ 3.5155e+00  2.3770e+00  1.2385e+00  1.0000e-01 -3.5000e-03 -1.0700e-01]
 [ 4.6540e+00  3.5155e+00  2.3770e+00  1.2385e+00  1.0000e-01 -3.5000e-03]
 [ 5.7925e+00  4.6540e+00  3.5155e+00  2.3770e+00  1.2385e+00  1.0000e-01]]
-----------------
[[1.90559975e+12 1.05973894e+13 5.89340238e+13 3.27742904e+14
  1.82263833e+15 1.01360257e+16]
 [1.21113472e+04 1.90559975e+12 1.05973894e+13 5.89340238e+13
  3.27742904e+14 1.82263833e+15]
 [7.69756242e-05 1.21113472e+04 1.90559975e+12 1.05973894e+13
  5.89340238e+13 3.27742904e+14]
 [4.89231019e-13 7.69756242e-05 1.21113472e+04 1.90559975e+12
  1.05973894e+13 5.89340238e+13]
 [3.10938680e-21 4.89231019e-13 7.69756242e-05 1.21113472e+04
  1.90559975e+12 1.05973894

In [5]:
# function to initialize the lattice
'''
Initialize the lattice
1 : atom
0 : vacancy or vacumm
2 more layer to make vacuum
'''
def init_3d_lattice(height, width, depth):
    lattice = np.ones((height+2, width, depth), dtype=int)
    lattice[0, :, :] = 0
    lattice[-1, :, :] = 0

    return lattice

In [6]:
# test
test_lattice = init_3d_lattice(4, 5, 6)
test_lattice[1, 1, 1] = 0
print(test_lattice[1])

[[1 1 1 1 1 1]
 [1 0 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]


In [7]:
# Function to plot the lattice of circles
def plot_lattice(ax, width, height, atom_radius, lattice):
    for x in range(width):
        for y in range(height+2):
            if lattice[y, x] == 1:
                circle = plt.Circle((x + 0.5, y + 0.5), atom_radius, color='blue')
                ax.add_artist(circle)
    ax.set_xlim(0, width)
    ax.set_ylim(0, height+2)
    ax.set_aspect('equal')
    ax.axis('off')
    plt.tight_layout(pad=0.5)

In [8]:
# Function to draw lattice on prompt
def draw_3d_lattice(lattice):
    height, width, depth = lattice.shape
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                if lattice[z, x, y] == 1:
                    print('●', end = '')
                else:
                    print(' ', end='')
            print('\n')
        print('\n')

In [9]:
# Function to save lattice
def save_lattice(lattice, output_filename):
    # Check if the file already exists, and if so, remove it
    try:
        with open(output_filename, 'x'):
            pass
    except FileExistsError:
        pass

    # Perform iterative saving
   
    with open(output_filename, 'ab') as file:
        np.savetxt(file, lattice, fmt='%d', delimiter=' ')
        file.write(b'\n')

# lattice to xyz
def save_matrix_as_xyz(matrix, output_filename):
    with open(output_filename, 'a') as output_file:
        num_atoms = int(np.sum(matrix))  # Calculate the number of Cu atoms (1s)

        # Write the number of atoms as the first line in the XYZ file
        output_file.write(f"{num_atoms}\n\n")

        # Iterate through the matrix to write the atom positions
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                for k in range(matrix.shape[2]):
                    if matrix[i, j, k] == 1:  # Cu atom
                        output_file.write(f"Cu {i} {j} {k}\n")

In [10]:
'''
function that finds every possible way
1. atom jumped from surface
2. return to previous position(jumped from surface)
3. other diffusion...
'''
def get_atoms_around_site(lattice, z, x, y, total=False):
    height, width, depth = lattice.shape
    # calculate bond number
    if z == height-1:
        down = 0
    else:
        down = lattice[z+1, x, y]
    if z == 0:
        up = 0
    else:
        up = lattice[z-1, x, y]
    # PBC at left and right
    right = lattice[z, x, (y+1)%depth]
    left = lattice[z, x, (y-1)%depth]
    forward = lattice[z, (x-1)%width, y]
    back = lattice[z, (x+1)%width, y]
    
    if total:
        return left+up+right+down+forward+back
    else:
        return up, down, left, forward, right, back

def find_candidate(lattice):
    global e_a
    global diffusion_rate
    height, width, depth = lattice.shape
    candidate_table = []
    diffusion_table = []
    motion_table = []
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                # find vacancy
                if lattice[z, x, y] == 0:
                    up, down, left, forward, right, back = get_atoms_around_site(lattice, z, x, y)
                    neighbor = left+up+right+down+forward+back
                    if neighbor:
                        '''
                        motion table
                        1 : up     2 : down     3 : left       4 : forward      5 : right       6 : back
                        '''
                        if up:
                            neighbor_of_up = get_atoms_around_site(lattice, z-1, x, y, True)
                            candidate_table.append((z-1, x, y))
                            motion_table.append(2)
                            diffusion_table.append(diffusion_rate[neighbor_of_up, neighbor-1])
                        if down:
                            neighbor_of_down = get_atoms_around_site(lattice, z+1, x, y, True)
                            candidate_table.append((z+1, x, y))
                            motion_table.append(1)
                            diffusion_table.append(diffusion_rate[neighbor_of_down, neighbor-1]) 
                        if left:
                            neighbor_of_left = get_atoms_around_site(lattice, z, x, (y-1)%depth, True)
                            candidate_table.append((z, x, (y-1)%depth))
                            motion_table.append(5)
                            diffusion_table.append(diffusion_rate[neighbor_of_left, neighbor-1])
                        if forward:
                            neighbor_of_forward = get_atoms_around_site(lattice, z, (x-1)%width, y, True)
                            candidate_table.append((z, (x-1)%width, y))
                            motion_table.append(6)
                            diffusion_table.append(diffusion_rate[neighbor_of_forward, neighbor-1]) 
                        if right:
                            neighbor_of_right = get_atoms_around_site(lattice, z, x, (y+1)%depth, True)
                            candidate_table.append((z, x, (y+1)%depth))
                            motion_table.append(3)
                            diffusion_table.append(diffusion_rate[neighbor_of_right, neighbor-1])
                        if back:
                            neighbor_of_back = get_atoms_around_site(lattice, z, (x+1)%width, y, True)
                            candidate_table.append((z, (x+1)%width, y))
                            motion_table.append(4)
                            diffusion_table.append(diffusion_rate[neighbor_of_back, neighbor-1]) 
    
    return candidate_table, diffusion_table, motion_table

In [11]:
# test
test_lattice[1, 1, 2] = 0
# print(test_lattice[0])
# print(test_lattice[1])
# print(test_lattice[2])
get_atoms_around_site(test_lattice, 2, 1, 2)
# draw_3d_lattice(test_lattice)

(0, 1, 1, 1, 1, 1)

In [12]:
# KMC function
def diffuse_one_step(lattice, print_out=False):
    global time_elapsed
    width, depth = lattice.shape[1], lattice.shape[2]

    cand, dif, motion = find_candidate(lattice)
    dif = np.array(dif)

    total_dif = np.sum(dif)
    
    # pick 1
    u = np.random.uniform(low=1e-6, high=1)
    u_time = np.random.uniform(low=1e-6, high=1)
    cum_dif = np.cumsum(dif)

    chosen_idx = np.argwhere(u*total_dif < cum_dif)[0][0]

    # print information
    if print_out:
        print(f'total_diff : {total_dif}')
        print(f'chosen : {cand[chosen_idx]} atom')   
    # print(f'motion : {motion[chosen_idx]}')
    # print(cand[chosen_idx])
    # print(motion[chosen_idx])

    # change the lattice
    z, x, y = cand[chosen_idx][0], cand[chosen_idx][1], cand[chosen_idx][2]

    # get motion
    if motion[chosen_idx] == 1:
        lattice[z, x, y] = 0
        lattice[z-1, x, y] = 1
        if print_out:
            print(f'go up')   
    if motion[chosen_idx] == 2:
        lattice[z, x, y] = 0
        lattice[z+1, x, y] = 1
        if print_out:
            print(f'go down')   
    if motion[chosen_idx] == 3:
        lattice[z, x, y] = 0
        lattice[z, x, (y-1)%depth] = 1
        if print_out:
            print(f'go left')   
    if motion[chosen_idx] == 4:
        lattice[z, x, y] = 0
        lattice[z, (x-1)%width, y] = 1
        if print_out:
            print(f'go forward')   
    if motion[chosen_idx] == 5:
        lattice[z, x, y] = 0
        lattice[z, x, (y+1)%depth] = 1
        if print_out:
            print(f'go right')   
    if motion[chosen_idx] == 6:
        lattice[z, x, y] = 0
        lattice[z, (x+1)%width, y] = 1
        if print_out:
            print(f'go back')   
    
    # time update
    delta_t = -np.log(u_time)/total_dif
    # print(f'delta_t : {delta_t}')
    time_elapsed += delta_t

## Simulation
---

In [14]:
# Parameters
# Size of the lattice (height x width)
width = 10
depth = 5  
height = 3

# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
'''
bond_energy = 2.07 
temperature = 700
e0 = 0.1

# site energy, e_(bond number)
e_site = np.zeros(6)
for i in range(6):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((6, 6))
for i in range(6):
    for j in range(6):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((6, 6))
for i in range(6):
    for j in range(6):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   
  
steps = 100
time_elapsed = 0

In [15]:
print(diffusion_rate)

[[1.90559975e+12 1.05973894e+13 5.89340238e+13 3.27742904e+14
  1.82263833e+15 1.01360257e+16]
 [1.21113472e+04 1.90559975e+12 1.05973894e+13 5.89340238e+13
  3.27742904e+14 1.82263833e+15]
 [7.69756242e-05 1.21113472e+04 1.90559975e+12 1.05973894e+13
  5.89340238e+13 3.27742904e+14]
 [4.89231019e-13 7.69756242e-05 1.21113472e+04 1.90559975e+12
  1.05973894e+13 5.89340238e+13]
 [3.10938680e-21 4.89231019e-13 7.69756242e-05 1.21113472e+04
  1.90559975e+12 1.05973894e+13]
 [1.97622103e-29 3.10938680e-21 4.89231019e-13 7.69756242e-05
  1.21113472e+04 1.90559975e+12]]


In [16]:
'''
3d simulation
It's hard to draw 3d lattice on prompt...
'''
# parameter tuning
steps = 10000
width = 100
depth = 100
height = 6
time_elapsed = 0
save_file_name = '100_100_10_st10000.xyz'
lattice_depo = []

lattice = init_3d_lattice(height, width, depth)
# lattice_depo.append(lattice.copy())
save_matrix_as_xyz(lattice, save_file_name)
# save_lattice(lattice, save_file_name)
atom_num = lattice.sum()
real_time_start = time.time()
for i in range(1, steps+1):
    diffuse_one_step(lattice)
    save_matrix_as_xyz(lattice, save_file_name)
    # lattice_depo.append(lattice.copy())
    if i % 1000 == 0:
        real_time_check = time.time()
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed in simul: {time_elapsed} s')
        print(f'Real time elapsed : {real_time_check-real_time_start} s')
    
    if lattice.sum() != atom_num:
        print('error!')
        break
    # time.sleep(0.2)



---------------- step 1000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 68.4084997177124 s
---------------- step 2000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 137.6679515838623 s
---------------- step 3000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 205.94603085517883 s
---------------- step 4000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 274.69613766670227 s
---------------- step 5000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 342.7804205417633 s
---------------- step 6000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 412.68895745277405 s
---------------- step 7000 ---------------------
Time elapsed in simul: 4.200913250958478e+26 s
Real time elapsed : 481.3501148223877 s
---------------- step 8000 -------------------

In [ ]:
# save to xyz file directly
# for lat in lattice_depo:
#     save_matrix_as_xyz(lat, save_file_name)

# Start with vacancy
---

In [21]:
'''
start simulation with one vacancy
'''
# parameter tuning
steps = 100
width = 100
height = 10
time_elapsed = 0
'-----------------------------------------------------------------'


# initialize
lattice = init_lattice(width, height)

# make vacancy
lattice[5, 50] = 0
for i in range(1, steps+1):
    print(f'---------------- step {i} ---------------------')
    diffuse_one_step(lattice, True)
    draw_lattice(lattice)
    print(f'Time elapsed : {time_elapsed} s')
    time.sleep(0.4)
    
    os.system('clear')
    clear_output(wait=True)

---------------- step 100 ---------------------
total_diff : 417930372361.77185
chosen : (10, 67) atom
go left
                                                                                                    

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●

In [47]:
# vacancy + no visualize
# parameter tuning
steps = 10000
width = 300
height = 20
time_elapsed = 0
'-----------------------------------------------------------------'

lattice = init_lattice(width, height)

# make vacancy
lattice[10, 150] = 0

atom_num = lattice.sum()
for i in range(1, steps+1):
    diffuse_one_step(lattice)
    if i % 1000 == 0:
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed : {time_elapsed} s')
        if lattice.sum() != atom_num:
            print('error!')
            break

---------------- step 1000 ---------------------
Time elapsed : 2.5593605100587533e-10 s
---------------- step 2000 ---------------------
Time elapsed : 5.29326820110763e-10 s
---------------- step 3000 ---------------------
Time elapsed : 7.922695829227711e-10 s
---------------- step 4000 ---------------------
Time elapsed : 1.0562340177351715e-09 s
---------------- step 5000 ---------------------
Time elapsed : 1.3173568931482398e-09 s
---------------- step 6000 ---------------------
Time elapsed : 1.5835682176808373e-09 s
---------------- step 7000 ---------------------
Time elapsed : 1.8564033993476096e-09 s
---------------- step 8000 ---------------------
Time elapsed : 2.1282875956763153e-09 s
---------------- step 9000 ---------------------
Time elapsed : 2.3835161816815982e-09 s
---------------- step 10000 ---------------------
Time elapsed : 2.6584171177304157e-09 s


# step by step check

In [115]:
# init
height = 5
width = 10
depth = 20
lattice = init_3d_lattice(height, width, depth)
print(lattice.sum())

1000


In [116]:
# step1
diffuse_one_step(lattice, True)

total_diff : 7.904884116549141e-27
chosen : (5, 1, 12) atom
go up


In [124]:
lattice[5].sum()

199

In [72]:
# step2
diffuse_one_step(lattice, True)

In [77]:
cand, dif, motion = find_candidate(lattice)

In [67]:
dif = np.array(dif)
total_dif = np.sum(dif)
u_time = np.random.uniform(low=1e-6, high=1)
delta_t = -np.log(u_time)/total_dif
print(delta_t)

3150303124.857064
